In [17]:
import numpy as np
from scipy.linalg import expm
from qiskit import *

$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$ 

# 1) Density Matrix - Covariance Matrix

We are given a matrix $\sigma_{2} = \begin{pmatrix} \alpha_{1} && \alpha \\ \alpha && \alpha_{2} \end{pmatrix}$ and we normalize it respect to its trace: 

$$ \rho_{2} = \frac{\sigma_{2}}{tr(\sigma_{2})} = \begin{pmatrix} \frac{\alpha_{1}}{\alpha_{1}+\alpha_{2}} && \frac{\alpha}{\alpha_{1}+\alpha_{2}} \\ \frac{\alpha}{\alpha_{1}+\alpha_{2}} && \frac{\alpha_{2}}{\alpha_{1}+\alpha_{2}} \end{pmatrix} $$

In [6]:
sigma2 = np.array([(1.89,0.97), (0.97, 1.06)])
rho2 = sigma2 /np.matrix.trace(sigma2)

We now calculate with numpy the eigenvalues and the eigenvectors using a classical algorithm:

In [10]:
eigenvalues,(eigenvector1, eigenvector2)= np.linalg.eig(rho2)

In [13]:
print(eigenvalues)

[0.85764318 0.14235682]


# 2) Calculate the unitary matrix gate

We know that the matrix could be turn into a unitary matrix by taking the exponential imaginary:

$$ U_{\rho_{2}} = e^{2\pi i \rho_{2}} $$ 

In [22]:
unitary_matrix = expm(2*1j*np.pi*rho2)

In [23]:
print(unitary_matrix)

[[ 6.25944518e-01-0.30675878j -4.26408010e-16-0.71700245j]
 [-4.90557316e-16-0.71700245j  6.25944518e-01+0.30675878j]]
